<a href="https://colab.research.google.com/github/dvschultz/stylegan2-ada-pytorch/blob/main/SG2_ADA_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN2-ADA-PyTorch

---

If you find this notebook useful, consider signing up for my [Patreon](https://www.patreon.com/bustbright) or [YouTube channel](https://www.youtube.com/channel/UCaZuPdmZ380SFUMKHVsv_AA/join). You can also send me a one-time payment on [Venmo](https://venmo.com/Derrick-Schultz).

In [1]:
# mount Google Drive on the runtime
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
# create a symbolic link to a working directory
!ln -s /content/gdrive/My\ Drive/Colab\ Notebooks/monixypAI /mydrive

# navigate to the working directory
%cd /mydrive

ln: failed to create symbolic link '/mydrive/monixypAI': File exists
/content/gdrive/My Drive/Colab Notebooks/monixypAI


## Setup

Let’s start by checking to see what GPU we’ve been assigned. Ideally we get a V100, but a P100 is fine too. Other GPUs may lead to issues.

In [3]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-6f0394f7-34db-0eb7-56b0-8b54b15c5118)


Next let’s connect our Google Drive account. This is optional but highly recommended.

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

## Install repo

The next cell will install the StyleGAN repository in Google Drive. If you have already installed it it will just move into that folder. If you don’t have Google Drive connected it will just install the necessary code in Colab.

In [5]:
import os
if os.path.isdir("/mydrive/stylegan2-ada-pytorch"):
    %cd "/mydrive/stylegan2-ada-pytorch/"
# elif os.path.isdir("/mydrive/"):
#     #install script
#     %cd "/mydrive/"
#     !mkdir colab-sg2-ada-pytorch
#     %cd colab-sg2-ada-pytorch
#     !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
#     %cd stylegan2-ada-pytorch
#     !mkdir downloads
#     !mkdir datasets
#     !mkdir pretrained
#     !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
# else:
elif os.path.isdir("/mydrive/"):
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip install ninja

/content/gdrive/My Drive/Colab Notebooks/monixypAI/stylegan2-ada-pytorch


You probably don’t need to run this, but this will update your repo to the latest and greatest.

In [6]:
%cd "/mydrive/stylegan2-ada-pytorch/"
!git config --global user.email 'adamsiemaszkiewicz@gmail.com'
!git config --global user.name 'adamsiemaszkiewicz'
!git fetch origin
!git pull
!git checkout origin/main -- train.py

/content/gdrive/My Drive/Colab Notebooks/monixypAI/stylegan2-ada-pytorch
Already up to date.


## Dataset Preparation

Upload a .zip of square images to the `datasets` folder. Previously you had to convert your model to .tfrecords. That’s no longer needed :)

### Resize images

In [7]:
# !python dataset_tool.py --help 

!python /mydrive/stylegan2-ada-pytorch/dataset_tool.py \
      --source /mydrive/images/ --dest /mydrive/images_out/ \
      --transform=center-crop-wide --width=512 --height=512

Error: --dest folder must be empty


In [8]:
# !python /mydrive/stylegan2-ada-pytorch/train.py --help

## Train model

Below are a series of variables you need to set to run the training. You probably won’t need to touch most of them.

* `dataset_path`: this is the path to your .zip file
* `resume_from`: if you’re starting a new dataset I recommend `'ffhq1024'` or `'./pretrained/wikiart.pkl'`
* `mirror_x` and `mirror_y`: Allow the dataset to use horizontal or vertical mirroring.

In [11]:
#required: definitely edit these!
dataset_path = '/mydrive/images/downscaled.zip'
resume_from = '/mydrive/stylegan2-ada-pytorch/pretrained/FreaGAN-10k.pt'
aug_strength = 0.0
train_count = 0
mirror_x = True

#broken, don't use for now :(
#mirror_y = False

#optional: you might not need to edit these
gamma_value = 50.0
augs = 'bg'
config = '11gb-gpu'
# config = 'paper512'
snapshot_count = 4

In [13]:
!python /mydrive/stylegan2-ada-pytorch/train.py --gpus=1 --cfg=$config \
        --metrics=None --outdir=./results --data=$dataset_path \
        --snap=$snapshot_count --augpipe=$augs \
        --initstrength=$aug_strength --gamma=$gamma_value \
        --mirror=$mirror_x --mirrory=False --nkimg=$train_count


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 4,
  "network_snapshot_ticks": 4,
  "metrics": [],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/mydrive/images/downscaled.zip",
    "use_labels": false,
    "max_size": 144,
    "xflip": true,
    "resolution": 512
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 8
    },
    "synthesis_kwargs": {
      "channel_base": 32768,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 8
    },
    "channel_base": 32768,
    "channel_max": 512,
    "num_fp16_re

### Resume Training

Once Colab has shutdown, you’ll need to resume your training. Reset the variables above, particularly the `resume_from` and `aug_strength` settings.

1. Point `resume_from` to the last .pkl you trained (you’ll find these in the `results` folder)
2. Update `aug_strength` to match the augment value of the last pkl file. Often you’ll see this in the console, but you may need to look at the `log.txt`. Updating this makes sure training stays as stable as possible.
3. You may want to update `train_count` to keep track of your training progress.

Once all of this has been reset, run that variable cell and the training command cell after it.

## Testing/Inference

TK

In [10]:
https://github.com/dvschultz/ml-art-colabs
https://github.com/dvschultz/stylegan2-ada-pytorch
https://www.youtube.com/watch?v=14JBICMUGfA
https://github.com/NVlabs/stylegan2-ada-pytorch